# Implement singleton design pattern in python

**Singleton is a creational design pattern** that lets you ensure that `a class has only one instance, while providing a global access point to this instance`.

The Singleton pattern solves two problems at the same time
- Ensure that a class has just a single instance
- Provide a global access point to that instance.

> But it violates the [Single Responsibility Principle](https://en.wikipedia.org/wiki/Single-responsibility_principle)

## 1. General solution

All implementations of the Singleton have these two steps in common:

- Make the default constructor private, to prevent other objects from using the new operator with the Singleton class.
- Create a static creation method that acts as a constructor. Under the hood, this method calls the private constructor to create an object and saves it in a static field. All following calls to this method return the cached object.

If your code has access to the Singleton class, then it’s able to call the Singleton’s static method. So whenever that method is called, the same object is always returned.

## 2. Python implementation

The Singleton class can be implemented in different ways in Python. Some possible methods include:
- base class,
- decorator,
- metaclass.

We will use the metaclass because it is best suited for this purpose. We also added a thread lock to make the `singleton thread safe`.

In [ ]:
from threading import Lock, Thread


class MySingletonMeta(type):
    """
    This is a thread-safe implementation of Singleton.
    """

    _instances = {}

    _lock: Lock = Lock()
    """
    We now have a lock object that will be used to synchronize threads during
    first access to the Singleton.
    """
    # The __call__ method enables Python programmers to write classes where the instances
    # behave like functions and can be called like a function.
    # When the instance is called as a function; if this method is defined, x(arg1, arg2, ...)
    # is a shorthand for x.__call__(arg1, arg2, ...).
    def __call__(cls, *args, **kwargs):
        """
        Possible changes to the value of the `__init__` argument do not affect
        the returned instance.
        """
        # Now, imagine that the program has just been launched. Since there's no
        # Singleton instance yet, multiple threads can simultaneously pass the
        # previous conditional and reach this point almost at the same time. The
        # first of them will acquire lock and will proceed further, while the
        # rest will wait here.
        with cls._lock:
            # The first thread to acquire the lock, reaches this conditional,
            # goes inside and creates the Singleton instance. Once it leaves the
            # lock block, a thread that might have been waiting for the lock
            # release may then enter this section. But since the Singleton field
            # is already initialized, the thread won't create a new object.
            if cls not in cls._instances:
                instance = super().__call__(*args, **kwargs)
                cls._instances[cls] = instance
        return cls._instances[cls]

After the creation of the MetaClass, we can now write the singleton class

In [ ]:
class UserSession(metaclass=MySingletonMeta):
    def __init__(self, lastName: str, firstName: str):
        # we will use full name to show that the singleton really works
        self.fullName = f"{firstName} {lastName}"
        self.connection = self.connect()

    def connect(self):
        # activate user session
        print(f"User {self.fullName} has connected.")
        return True


# this function can help us to test the singleton class in a multi thread env
def createUserSession(firstName, lastName):
    session = UserSession(firstName, lastName)
    print(session.fullName)

Use the below main method to test the singleton

In [ ]:
def main():
    # test it in a single thread
    createUserSession("toto", "Liu")
    createUserSession("t1t1", "Liu")

    # test it with multi thread
    # sessionThread1 = threading.Thread(target=createUserSession, args=("toto", "liu"))
    # sessionThread2 = threading.Thread(target=createUserSession, args=("titi", "liu"))
    # sessionThread1.start()
    # sessionThread2.start()


if __name__ == "__main__":
    main()

You can notice there is only one instance of the user session, even you create a new session with another username, The instance is always the first instance that you have created.

You can find the full example in `MySingleton.py`